# Configurations

In [ ]:
#@title AWS Credentials Setup

AWS_ACCESS_KEY_ID = '' #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = '' #@param {type:"string"}
CLUSTER_NAME = '' #@param {type:"string"}
REGION = '' #@param {type:"string"}

NAMESPACE = 'hello-docker' #@param {type:"string"}
DOCKER_IMAGE = 'docker.io/uniquetrij/hello-docker:screened' #@param {type:"string"}

SERVER_PORT = '8000' #@param {type:"string"}
SPRING_SECURITY_OAUTH2_CLIENT_PROVIDER_OIDC_ISSUER_URI = '' #@param {type:"string"}
SPRING_DATASOURCE_URL = '' #@param {type:"string"}
SPRING_DATASOURCE_USERNAME = '' #@param {type:"string"}
SPRING_DATASOURCE_PASSWORD = '' #@param {type:"string"}





# Setup

In [ ]:
!pip install awscli # !apt update && apt install awscli -y
!aws --version

In [ ]:
!curl -LO https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl
!chmod +x ./kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!source ~/.bashrc
!kubectl version

In [ ]:
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add - 
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable" 
!sudo apt install docker-ce -y
# !sudo apt install docker.io -y
!docker -v

In [ ]:
!aws configure set aws_access_key_id $AWS_ACCESS_KEY_ID
!aws configure set aws_secret_access_key $AWS_SECRET_ACCESS_KEY
!aws configure set region $REGION
!aws sts get-caller-identity
!aws eks --region $REGION update-kubeconfig --name $CLUSTER_NAME
!kubectl get namespaces --kubeconfig ~/.kube/config

In [ ]:
ACCOUNT = !aws sts get-caller-identity --query "Account" --output text
ACCOUNT = ACCOUNT[0]
ECR = !echo $(echo $ACCOUNT).dkr.ecr.$(echo $REGION).amazonaws.com
ECR = ECR[0]

!aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ECR

In [ ]:
from IPython.core.magic import register_line_cell_magic
import re
BRACES='{}'
@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        cell = re.sub(r'(\$\{)([a-zA-Z0-9_-]+)(\})' , r'{\2}', cell)
        cell = re.sub(r'(\$)([a-zA-Z0-9_-]+)' , r'{\2}', cell)
       
        cell = re.sub(r'\{\}' , '{BRACES}', cell)
        f.write(cell.format(**globals()))

# YAML

In [ ]:
%%writetemplate namespace.yml
apiVersion: v1
kind: Namespace
metadata:
  name: $NAMESPACE

In [ ]:
!cat namespace.yml

In [ ]:
!kubectl apply -f namespace.yml
!kubectl get namespaces #| grep $NAMESPACE

In [ ]:
%%writetemplate configmap.yml
kind: ConfigMap
apiVersion: v1
metadata:
  name: ${NAMESPACE}-configmap
  namespace: $NAMESPACE
data:
  SERVER_PORT: "$SERVER_PORT"

In [ ]:
!cat configmap.yml

In [ ]:
!kubectl apply -f configmap.yml
!kubectl get configmap -n $NAMESPACE #| grep $NAMESPACE

In [ ]:
%%writetemplate service.yml
apiVersion: v1
kind: Service
metadata:
  name: $NAMESPACE
  namespace: $NAMESPACE
  labels:
    app: $NAMESPACE
spec:
  selector:
    app: $NAMESPACE
  ports:
    - protocol: TCP
      port: $SERVER_PORT
  sessionAffinity: ClientIP
  type: ClusterIP

In [ ]:
!cat service.yml

In [ ]:
!kubectl apply -f service.yml
!kubectl get service -n $NAMESPACE #| grep $NAMESPACE

In [ ]:
%%writetemplate deployment.yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: $NAMESPACE
  namespace: $NAMESPACE
  labels:
    app: $NAMESPACE
spec:
  replicas: 1
  selector:
    matchLabels:
      app: $NAMESPACE
  strategy: {}
  template:
    metadata:
      labels:
        app: $NAMESPACE
    spec:
      containers:
        - name: $NAMESPACE
          image: $DOCKER_IMG
          imagePullPolicy: Always
          ports:
            - name: http
              containerPort: $SERVER_PORT
      restartPolicy: Always
      serviceAccountName: ''

In [ ]:
!cat deployment.yml

In [ ]:
!kubectl apply -f deployment.yml
!kubectl get deployment -n $NAMESPACE #| grep $NAMESPACE

In [ ]:
%%writetemplate ingress.yml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: ${NAMESPACE}-ingress
  namespace: $NAMESPACE
  annotations:
    kubernetes.io/ingress.class: "nginx"
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
    - http:
        paths:
          - path: /${NAMESPACE}
            pathType: ImplementationSpecific
            backend:
              service:
                name: ${NAMESPACE}
                port:
                  number: ${SERVER_PORT}





In [ ]:
!cat ingress.yml

In [ ]:
!kubectl apply -f ingress.yml
!kubectl get ingress -n $NAMESPACE #| grep $NAMESPACE

In [ ]:
!kubectl describe ingress -n $NAMESPACE

In [ ]:
!aws ecr create-repository --repository-name $NAMESPACE 

In [ ]:
%%bash -s $ECR $NAMESPACE $DOCKER_IMG
set -x
dockerd -b none --iptables=0 -l fatal &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
echo y | docker system prune -a
docker pull $3 | grep "Digest:" | cut -f2 -d " " > container_digest
cat container_digest
IMG=$(docker images --digests | grep $(cat container_digest) | sed -Ee 's/\s+/ /g' | cut -f4 -d " ")
echo $IMG

# # IMG=$(docker images | grep $3 | awk '{ print $3 }')
# # TAG=$(docker images | grep $3 | awk '{ print $2 }')

docker tag $IMG $1/$2
docker push $1/$2
kill $(jobs -p)

In [ ]:
# !aws ecr create-repository --repository-name campaigns/flow-nodes-summary
!aws ecr describe-images --repository-name $NAMESPACE
# !aws ecr describe-repositories --region $REGION

# Clean Namespace Resources

In [ ]:
!kubectl delete all --all -n $NAMESPACE
!kubectl get all -n $NAMESPACE

In [ ]:
!kubectl get pods -n $NAMESPACE | grep $NAMESPACE | awk '{ print $1 }'

In [ ]:
! kubectl exec --stdin --tty hello-docker /bin/bash

In [ ]:
#! curl hello-docker.hello-docker.svc.cluster.local:8000/hello-docker/